In [34]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_csv('lane_data_1.csv')
df = df[df['Location'] == 'us-101']

In [50]:
df

,Unnamed: 0,v_Vel,Lane_ID,Location,Global_Time
8242843,8242843,40.00,4,us-101,1118846979700
5805738,5805738,40.00,4,us-101,1118846979800
7908571,7908571,40.00,4,us-101,1118846979900
459432,459432,40.00,4,us-101,1118846980000
5527553,5527553,40.00,4,us-101,1118846980100
...,...,...,...,...,...
2234970,2234970,45.41,3,us-101,1118847084800
1740711,1740711,61.85,3,us-101,1118847084800
8281136,8281136,10.00,1,us-101,1118847084800
1568462,1568462,58.91,4,us-101,1118847084800


In [36]:
df.sort_values('Global_Time', inplace=True)

In [40]:
lane_cnt = max(df['Lane_ID'])
print(lane_cnt)

rows = len(df['Lane_ID'])
df.head()


8


,Unnamed: 0,v_Vel,Lane_ID,Location,Global_Time
8242843,8242843,40.0,4,us-101,1118846979700
5805738,5805738,40.0,4,us-101,1118846979800
7908571,7908571,40.0,4,us-101,1118846979900
459432,459432,40.0,4,us-101,1118846980000
5527553,5527553,40.0,4,us-101,1118846980100


In [41]:
df = df[0:100000]

In [42]:
def get_initial_matrix():
    matrix_out_degree = np.zeros((lane_cnt, lane_cnt))
    for i in range(1, rows):
        prev_lane_id = int(df.iloc[i - 1][2])
        present_lane_id = int(df.iloc[i][2])
        # print(present_lane_id, present_lane_id)
        # if i == 100:
        #     break
        matrix_out_degree[prev_lane_id - 1][present_lane_id - 1] += 1
    return matrix_out_degree


matrix = get_initial_matrix()

In [43]:
matrix

array([[4.700e+03, 4.775e+03, 4.139e+03, 3.794e+03, 4.012e+03, 3.970e+02,
        9.400e+01, 4.700e+01],
       [4.678e+03, 4.379e+03, 3.955e+03, 3.631e+03, 4.018e+03, 4.410e+02,
        9.600e+01, 4.000e+01],
       [4.115e+03, 3.936e+03, 3.395e+03, 3.152e+03, 3.546e+03, 3.800e+02,
        7.200e+01, 3.500e+01],
       [3.782e+03, 3.696e+03, 3.179e+03, 2.835e+03, 3.144e+03, 3.250e+02,
        6.500e+01, 4.200e+01],
       [4.063e+03, 3.935e+03, 3.498e+03, 3.209e+03, 3.339e+03, 3.560e+02,
        8.300e+01, 4.700e+01],
       [4.570e+02, 4.130e+02, 3.510e+02, 3.380e+02, 3.430e+02, 3.000e+01,
        8.000e+00, 2.000e+00],
       [9.800e+01, 7.000e+01, 7.400e+01, 7.700e+01, 8.600e+01, 1.200e+01,
        0.000e+00, 1.000e+00],
       [6.600e+01, 3.400e+01, 4.000e+01, 3.100e+01, 4.200e+01, 1.000e+00,
        0.000e+00, 0.000e+00]])

In [44]:
def get_transition_matrix(mat):
    ret_mat = np.zeros((lane_cnt, lane_cnt))
    for row in range(lane_cnt):
        sum = 0
        for col in range(lane_cnt):
            sum += mat[row][col]
        for col in range(lane_cnt):
            ret_mat[row][col] = mat[row][col] / sum

    return ret_mat


transition_matrix = get_transition_matrix(matrix)

In [51]:
transition_matrix

array([[0.21404499, 0.21746061, 0.18849622, 0.17278441, 0.18271245,
        0.01807997, 0.0042809 , 0.00214045],
       [0.22026556, 0.20618702, 0.18622281, 0.17096713, 0.18918919,
        0.02076467, 0.0045202 , 0.00188342],
       [0.22086845, 0.2112608 , 0.18222318, 0.1691804 , 0.19032795,
        0.02039611, 0.00386453, 0.00187859],
       [0.22158425, 0.21654558, 0.18625498, 0.1661003 , 0.18420436,
        0.01904148, 0.0038083 , 0.00246075],
       [0.21926606, 0.21235834, 0.18877496, 0.17317863, 0.18019428,
        0.01921209, 0.00447922, 0.00253643],
       [0.23532441, 0.21266735, 0.1807415 , 0.17404737, 0.17662204,
        0.01544799, 0.00411946, 0.00102987],
       [0.23444976, 0.16746411, 0.17703349, 0.18421053, 0.20574163,
        0.02870813, 0.        , 0.00239234],
       [0.30841121, 0.1588785 , 0.18691589, 0.14485981, 0.19626168,
        0.0046729 , 0.        , 0.        ]])

In [45]:
def get_initial_state_probability(mat):
    ret_mat = np.zeros(lane_cnt)
    total_sum = np.sum(mat)
    for i in range(lane_cnt):
        ret_mat[i] = np.sum(mat[i]) / total_sum
    return ret_mat


initial_state_prob = get_initial_state_probability(matrix)

In [52]:
initial_state_prob

array([0.2195822 , 0.21238212, 0.18631186, 0.17068171, 0.18530185,
       0.01942019, 0.00418004, 0.00214002])

In [46]:
speed_division = 3

In [54]:
def speed_range(speed):
    if speed <= 40:
        return 0
    if speed <= 60:
        return 1
    return 2


def get_emi_matrix(mat):
    matrix_out_degree = np.zeros((lane_cnt, speed_division))
    for i in range(1, rows):
        present_lane_id = int(df.iloc[i][2])
        speed = df.iloc[i][1]
        matrix_out_degree[present_lane_id - 1][speed_range(speed)] += 1
    return matrix_out_degree


emi_mat = get_emi_matrix(matrix)

In [55]:
emi_mat

array([[13041.,  8098.,   820.],
       [ 9306., 10715.,  1217.],
       [ 7739.,  9578.,  1314.],
       [ 4287., 11347.,  1433.],
       [ 4815., 12501.,  1214.],
       [   61.,  1040.,   841.],
       [   78.,   265.,    75.],
       [    0.,    71.,   143.]])

In [56]:
def get_emission_probability(mat):
    ret_mat = np.zeros((lane_cnt, speed_division))
    for row in range(lane_cnt):
        for col in range(speed_division):
            ret_mat[row][col] = mat[row][col] / np.sum(mat[row])
    return ret_mat


emission_probability = get_emission_probability(emi_mat)

In [57]:
np.savetxt("tm.txt", transition_matrix, delimiter=" ")
np.savetxt("em.txt", emission_probability, delimiter=" ")
np.savetxt("ini_p.txt", initial_state_prob, delimiter=" ")